# Imports

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
from sklearn.metrics import classification_report
import pandas
import numpy as np
import data_massage
import itertools
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

# tensorflow on my laptop seems to not obtain the correct amount of memory
# the GPU on it has 2G, but asking for 1300 was too much, 1000 worked, so there you go
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
          gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000)])

2022-12-11 13:03:37.507275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 13:03:37.676546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 13:03:37.679116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
metadata, full_data, induction_only = data_massage.load_data()
summarized_data = data_massage.summarize_data(metadata, induction_only)

lstm_data = summarized_data
features = sorted(["R_mean_mean", "Temp_mean", "Humidity_mean", "time_to_gas_max"])

hidden_layers = tuple(map(lambda p: (len(features) ** p), [3,4,5,4,3]))
feature_permutations = list(itertools.chain.from_iterable(map(lambda r: itertools.combinations(features, r), range(1, 1 + len(features)))))

In [8]:
results = {}

for perm in feature_permutations:
    feature_set = list(perm)
    lstm_train_x, lstm_test_x, lstm_train_y, lstm_test_y = train_test_split(
          lstm_data[feature_set]
        , lstm_data["class"]
        , test_size=0.2
        , stratify=lstm_data["class"]
        , random_state=7
    )

    label_encoder = LabelEncoder()
    label_encoder.fit(lstm_train_y)
    label_encoder.classes_
    lstm_train_y = label_encoder.transform(lstm_train_y)
    lstm_test_y = label_encoder.transform(lstm_test_y)
    
    model = Sequential()
    model.add(LSTM(units = lstm_train_x.shape[1], return_sequences = True, input_shape=(lstm_train_x.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1] ** 2, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1] ** 3, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1]))
    model.add(Dropout(0.2))
    model.add(Flatten())
    # Adding the output layer
    model.add(Dense(units = lstm_train_x.shape[1]))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

    # Fitting the RNN to the Training set
    model.fit(lstm_train_x, lstm_train_y, epochs = 30, batch_size = 4, verbose=False)
    
    #evaluation = model.evaluate(lstm_test_x, lstm_test_y)
    
    
    y_pred = model.predict(lstm_test_x, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    
    
    print(feature_set)
    print(classification_report(lstm_test_y, y_pred_bool))
    print("\n\n")
    
    #key = ','.join(feature_set)
    #results[key] = {"evaluation": model.evaluate(lstm_test_x, lstm_test_y)}
    
    #print(key)
    #print(results[key]["evaluation"])
    #print("\n\n\n")

1/1 [==============================] - 0s 477ms/step
['Humidity_mean']
              precision    recall  f1-score   support

           0       0.30      1.00      0.46         6
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         7

    accuracy                           0.30        20
   macro avg       0.10      0.33      0.15        20
weighted avg       0.09      0.30      0.14        20






/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 492ms/step
['R_mean_mean']
              precision    recall  f1-score   support

           0       0.30      1.00      0.46         6
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         7

    accuracy                           0.30        20
   macro avg       0.10      0.33      0.15        20
weighted avg       0.09      0.30      0.14        20






/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 1s 503ms/step
['Temp_mean']
              precision    recall  f1-score   support

           0       0.30      1.00      0.46         6
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         7

    accuracy                           0.30        20
   macro avg       0.10      0.33      0.15        20
weighted avg       0.09      0.30      0.14        20






...

/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 1s 513ms/step
['R_mean_mean', 'Temp_mean', 'time_to_gas_max']
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.35      1.00      0.52         7
           2       0.00      0.00      0.00         7

    accuracy                           0.35        20
   macro avg       0.12      0.33      0.17        20
weighted avg       0.12      0.35      0.18        20






/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 483ms/step
['Humidity_mean', 'R_mean_mean', 'Temp_mean', 'time_to_gas_max']
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         7
           2       0.35      1.00      0.52         7

    accuracy                           0.35        20
   macro avg       0.12      0.33      0.17        20
weighted avg       0.12      0.35      0.18        20






/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
